In [1]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from tensorflow.keras.layers import Dense  
from sensor.components.prepare_base_model import PrepareBaseModel
from sensor.config.configuration import Configuration

2025-06-16 17:10:19.313990: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-16 17:10:19.319350: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-16 17:10:19.333543: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750083019.354626  179433 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750083019.361192  179433 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1750083019.376820  179433 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

In [2]:
# Load preprocessed data
preprocessed_data_path = '/home/moraa-ontita/Documents/Machine-learning/Gas_Sensor/artifacts/preprocessed/notebook_preprocessed.csv'
data = pd.read_csv(preprocessed_data_path)



In [3]:
data.head()

,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,...,feature_120,feature_121,feature_122,feature_123,feature_124,feature_125,feature_126,feature_127,feature_128,target
0,0.676157,0.658295,0.722534,0.756207,0.727536,0.302161,0.071278,0.103377,0.649085,0.496269,...,0.131901,0.573114,0.500836,0.666088,0.553585,0.683751,0.429972,0.106052,0.135831,1
1,0.502643,0.516996,0.563249,0.629137,0.626112,0.605105,0.087501,0.114002,0.462891,0.361376,...,0.164178,0.329639,0.312260,0.515481,0.482845,0.612832,0.737153,0.163046,0.169744,1
2,0.674979,0.703586,0.696306,0.718099,0.693556,0.346455,0.052010,0.095141,0.678156,0.584185,...,0.125351,0.637336,0.652883,0.722456,0.593954,0.722695,0.412082,0.099673,0.126608,2
3,0.380613,0.390597,0.332772,0.515395,0.601823,0.803248,0.104949,0.117789,0.372752,0.272514,...,0.169882,0.241707,0.196775,0.210215,0.195645,0.360144,0.866871,0.183067,0.177099,2
4,0.730503,0.714337,0.772236,0.802591,0.751103,0.238953,0.063224,0.096749,0.705728,0.557537,...,0.113260,0.704145,0.633166,0.759772,0.586982,0.710484,0.287408,0.072757,0.119156,1


In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

# Separate features and target
X = data.drop('target', axis=1)
y = data['target']

# One-Hot Encoding the target variable
encoder = OneHotEncoder(sparse_output=False)  # Use sparse_output for newer versions
y_encoded = encoder.fit_transform(y.values.reshape(-1, 1))

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Check shapes
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)


(11128, 128) (11128, 6)
(2782, 128) (2782, 6)


In [6]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import load_model
import os

os.chdir("/home/moraa-ontita/Documents/Machine-learning/Gas_Sensor")

# Load the configuration
config = Configuration()
model_config = config.get_model_config()

# Load the prepared model
model_path = '/home/moraa-ontita/Documents/Machine-learning/Gas_Sensor/artifacts/prepared_model/gas_classification_model.keras'
prepare_model = PrepareBaseModel(config=model_config)

# Load the model from the saved path
prepare_model.model = load_model(model_path)

# Assuming X_train and y_train are defined and y_train is one-hot encoded
# Train the model
history = prepare_model.model.fit(X_train, y_train, 
                                   validation_split=0.2, 
                                   epochs=100, 
                                   batch_size=32, 
                                   callbacks=[EarlyStopping(patience=5, restore_best_weights=True)])


[2025-06-16 17:11:06,898: INFO: configuration: Reading configuration file from config/config.yaml]
[2025-06-16 17:11:06,902: INFO: configuration: Reading configuration file from param.yaml]
[2025-06-16 17:11:06,904: INFO: configuration: Model config data: {'input_shape': [128], 'save_dir': 'artifacts/prepared_model/', 'optimizer': 'adam', 'classification_loss': 'categorical_crossentropy', 'drift_loss': 'mean_squared_error', 'classification_metric': 'accuracy', 'drift_metric': 'mse'}, Type: <class 'dict'>]
[2025-06-16 17:11:06,905: INFO: prepare_base_model: PrepareBaseModel initialized with configuration.]


2025-06-16 17:11:06.911091: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
/home/moraa-ontita/Documents/Machine-learning/Gas_Sensor/.venv/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 18 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Epoch 1/100
279/279 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.1812 - loss: nan - val_accuracy: 0.1815 - val_loss: nan
Epoch 2/100
279/279 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.1872 - loss: nan - val_accuracy: 0.1815 - val_loss: nan
Epoch 3/100
279/279 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.1893 - loss: nan - val_accuracy: 0.1815 - val_loss: nan
Epoch 4/100
279/279 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.1872 - loss: nan - val_accuracy: 0.1815 - val_loss: nan
Epoch 5/100
279/279 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.1909 - loss: nan - val_accuracy: 0.1815 - val_loss: nan
Epoch 6/100
279/279 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.1849 - loss: nan - val_accuracy: 0.1815 - val_loss: nan
